In [1]:
import numpy as np
from scipy import optimize
import pykep

In [4]:
def get_rho(r_obs,r_hat, r):
    c = np.dot(r_obs,r_hat)
    return -c + np.sqrt(c**2 - np.linalg.norm(r_obs)**2 + r**2)

In [19]:
def func(r,R1,R2,hat1,hat2,t,mu=398600):
    rho1 = get_rho(R1,hat1,r)
    rho2 = get_rho(R2,hat2,r)
    r1 = R1 + rho1*hat1
    r2 = R2 + rho2*hat2
    return t - np.sqrt((r**3)/mu)*np.arccos(np.dot(r1,r2)/r**2)

In [22]:
def get_r(R1,R2,hat1,hat2, t):
    sol = optimize.fsolve(func,7000,args=(R1,R2,hat1,hat2,t))
    return sol


In [5]:
def get_target_r(r_obs,r_tar_hat, r):
    rho = get_rho(r_obs,r_tar_hat, r)
    return r_obs + rho*r_tar_hat

In [6]:
def propagate_based_on_elements(elements, t, kepler=True):
    r, v = pykep.par2ic(elements, mu=398600)
    if t > 0:
        rf,vf = pykep.propagate_lagrangian(r0 = r, v0 = v, tof = t,mu = 398600)
    else:
        rf, vf = r, v
    if kepler == False:  # return vectors
        return np.array(rf),np.array(vf)
    if kepler == True:  # return kepler elements
        return pykep.ic2par(rf,vf,mu = 398600)

In [26]:
def get_h(r1,r2):
    return np.cross(r1,r2)/np.linalg.norm(np.cross(r1,r2))

In [23]:
def get_v(h,r1,r, mu=398600):
    v_hat = np.cross(h,r1)/r
    return v_hat * np.sqrt(mu/r)

In [142]:
def circular_method(measurements, times, elements_obs):
    sol =[]
    all_r = []
    for i in range(len(measurements)):
        R1, V1 = propagate_based_on_elements(elements_obs,times[i],kepler=False)
        t1 = times[i]
        for j in range(len(measurements)):
            if j != i:
                tau = times[j] - t1
                if tau > 0:
                    R2, V2 = propagate_based_on_elements(elements_obs,times[j],kepler=False)
                    r = get_r(R1,R2,measurements[i],measurements[j],tau)
                    all_r.append(r)

    r = sum(all_r)/len(all_r)
    print("average r",r)
    targets_vel = [] 
    targets_pos = []
    for i in range(len(measurements)):
        r_obs, v_obs = propagate_based_on_elements(elements_obs,times[i],kepler=False)
        r_tar = get_target_r(r_obs, measurements[i],r)
        targets_pos.append(r_tar)
    
    h = get_h(targets_pos[0], targets_pos[-1])
    for i in range(len(measurements)):
        v_tar = get_v(h,targets_pos[i],r)
        targets_vel.append(v_tar)

    return targets_pos,targets_vel

https://apps.dtic.mil/sti/pdfs/AD1121954.pdf  -  
https://www.sciencedirect.com/science/article/pii/S0273117721005287#e0015  - another novel one
https://www.sciencedirect.com/science/article/pii/B9780123747785000052#b0160  - the book
https://www.researchgate.net/publication/254199485_Modifications_to_the_Gooding_Algorithm_for_Angles-Only_Initial_OrbitDetermination  - Cooding
file:///C:/Users/Admin/Downloads/Thesis_Bert_Van_den_Abbeele_v2.pdf  - Delft
https://conference.sdo.esoc.esa.int/proceedings/sdc8/paper/129/SDC8-paper129.pdf  - cooding useless?


In [169]:
def get_residual(estimated,measured):
    print("difference", measured-estimated)
    return np.arctan2(np.sin(measured-estimated),np.cos(measured-estimated))

In [43]:
def get_residuals(L,measurement):
    residuals = []
    for i in range(len(L)):
        residual = get_residual(L[i], measurement[i])
        residuals.append(residual)
    return residuals

In [111]:
def get_L(r,elements,t):
    r_obs, v_obs = propagate_based_on_elements(elements,t,kepler=False)
    L = r - r_obs
    print("L", L)
    return L/np.linalg.norm(L)

In [113]:
def iteration(r_vec,measurement,elements, t):
    L = get_L(r_vec,elements,t)
    residuals = get_residuals(L,measurement)
    return np.array(residuals)

In [244]:
def get_J(x,measurement,elements,t):
    eps = 1e-8
    J = np.zeros([3, 1], dtype = np.float64)

    mod = np.linalg.norm(x.copy())
    x1 = mod
    x2 = mod

    x1 += eps
    x2 -= eps
    x1 = x1*x/mod
    x2 = x2*x/mod

    f1 = iteration(x1,measurement,elements,t)
    f2 = iteration(x2,measurement,elements,t)
        
    J[ : , 0] = (f1 - f2) / (2 * eps)

    return J

In [204]:
def check_zeros(M):
    n = np.count_nonzero(M == 0)
    if n > 2:
        return True
    if n == 2:
        if M[0][0] == 0 and M[1][1] == 0:
            return False
        if M[0][1] == 0 and M[1][0] == 0:
            return False
        else:
            return True
    else:
        return False

In [222]:
def solve_eq(J1,J2,r,measurement, residuals, elements, t):
    H = np.hstack((J1,J2))
    
    W = np.diag([1,1,1])
    
    A = np.matmul(H.transpose(),W)
    
    A = np.matmul(A,H) 
    print("A",A)
    print("det A",np.linalg.det(A))
    if np.linalg.det(A) == 0 and check_zeros(A) is True:
        return 0,0
    if np.linalg.det(A) == 0:  # idk what to do here
        return 0,0
    else:
        residuals = residuals.transpose()
        b = np.matmul(H.transpose(),residuals)
        
        return np.linalg.solve(A,b)


In [183]:
def objective_func(sum, estimated, measurement):
    sum += (estimated[0] - measurement[0])**2 + (estimated[1] - measurement[1])**2 + (estimated[2] - measurement[2])**2
    return sum

In [126]:
def update(r, dr):
    norm = np.linalg.norm(r)
    unit = r/norm
    norm += dr
    return unit*norm

In [236]:
def method(measurements,times,elements_sat,tol=1e-18, k_max=40):
    # solve for initial guess
    targets_pos, targets_vel = circular_method(measurements,times,elements_sat)
    
    # set up the first iteration
    tau = times[-1] - times[0]
    r1 = targets_pos[0]
    r2 = targets_pos[-1]
    obj_last = None
    print("r1",r1)
    print("r2",r2)
    for k in range(k_max): 
        # solve Lambert
        l = pykep.lambert_problem(r1 = r1, r2 = r2, tof = tau, mu=398600)
        v1 = l.get_v1()[0]

        # calculate the Jacobians at the first and last measurements
        J1 = get_J(r1,measurements[0],elements_sat,times[0])
        J2 = get_J(r2,measurements[-1],elements_sat,times[-1])
        
        # set up for the corrections and objective func
        dr1_list = []
        dr2_list = []
        obj = 0

        for i in range(len(measurements)): 
            # propagate the solution in time
            print("at", i)
            TOF = times[i] - times[0]
            print(TOF)
            if TOF > 0:
                r, v = pykep.propagate_lagrangian(r0 = r1, v0 = v1, tof = TOF,mu = 398600)
                r,v = np.array(r), np.array(v)
            else:
                r, v = r1, v1
                
            print("r",r)

            # calculate the residuals at the current measurement
            residuals = iteration(r,measurements[i], elements_sat, times[i])
            print("individual residuals", residuals)
            # estimate the corrections
            dr1, dr2 = solve_eq(J1,J2,r,measurements[i],residuals, elements_sat,times[i])
            # append the corrections
            print("indiviudal corrections", dr1,dr2)
            dr1_list.append(dr1)
            dr2_list.append(dr2)
            L = get_L(r, elements_sat,times[i])
            print("measurement", measurements[i])
            print("residuals", residuals)
            # update the objective function
            obj = objective_func(obj,L,measurements[i])

        print("obj", obj)
        # check if the objective func is below tolerance
        if obj_last is not None and abs(obj - obj_last) < tol:
            print("tolerance reached")
            return r1,r2,tau
        
        # update the objective value for the next iteration
        obj_last = obj
        # take the average of all corrections
        dr1 = sum(dr1_list)/len(dr1_list)
        dr2 = sum(dr2_list)/len(dr2_list)
        print("corrections", dr1,dr2)
        # update r1 and r2 for the next iteration
        r1 = update(r1,dr1)
        r2 = update(r2,dr2)
        
    return None

In [246]:
elements_sat = [7350,0.0037296,np.radians(82.0394),np.radians(41.7294),np.radians(216.168),np.radians(0)]
elements_deb = [7500,0.01,np.radians(2),np.radians(10),np.radians(0),np.radians(45)]

times = 100*np.linspace(0,9,10)
measurements = []
for t in times:
    r_sat, v_sat = propagate_based_on_elements(elements_sat,t,kepler=False)
    r_deb, v_deb = propagate_based_on_elements(elements_deb,t,kepler=False)
    print("debris r", r_deb)
    print("debrs v",v_deb)
    L = r_deb - r_sat
    
    L_hat = L/np.linalg.norm(L)
    dec = np.arcsin(L_hat[2])
    
    measurements.append(L_hat)

debris r [4228.56970228 6127.1741877   185.07327648]
debrs v [-6.01369583  4.20788118  0.18117668]
debris r [3607.79069306 6517.76280628  202.27007379]
debrs v [-6.39176982  3.59783653  0.16248971]
debris r [2952.29292956 6845.63475412  217.52056149]
debrs v [-6.7075502   2.95459908  0.14228349]
debris r [2268.45082677 7107.79046301  230.68292969]
debrs v [-6.95824809  2.28460376  0.1207624 ]
debris r [1562.88376879 7301.88482331  241.63639003]
debrs v [-7.14175528  1.59449237  0.09814207]
debris r [ 842.38750464 7426.24413991  250.2821748 ]
debrs v [-7.25665351  0.89103904  0.07464687]
debris r [ 113.86504379 7479.875688    256.54428459]
debrs v [-7.30221526  0.18107698  0.05050739]
debris r [-615.74212126 7462.4700364   260.36998528]
debrs v [-7.27839656 -0.52857277  0.02595793]
debris r [-1339.52169744  7374.39639966   261.7300586 ]
debrs v [-7.18582237e+00 -1.23116940e+00  1.23409256e-03]
debris r [-2050.65648044  7216.69136047   260.61881385]
debrs v [-7.02576546 -1.92011674 -0.02

In [247]:
r1, r2, tof = method(measurements,times,elements_sat)
print(r1)
print(r2)
print(tof)
l = pykep.lambert_problem(r1=r1,r2=r2,tof=tof,mu=398600)
v =l.get_v1()[0]
pykep.ic2par(r1,v,mu=398600)

average r [7463.18648692]
r1 [4238.5171844  6139.85739193  190.46212971]
r2 [-2037.91489011  7176.09193425   223.06947446]
difference -6.661338147750939e-15
difference -7.827072323607354e-14
difference 1.965094753586527e-13
difference 6.5503158452884236e-15
difference 7.827072323607354e-14
difference -1.965094753586527e-13
difference 8.87900863943969e-14
difference -3.638200851696638e-13
difference 4.235500838944972e-13
difference -8.884559754562815e-14
difference 3.638200851696638e-13
difference -4.234390615920347e-13
at 0
0.0
r [4238.5171844  6139.85739193  190.46212971]
difference 1.1102230246251565e-16
difference 1.1102230246251565e-16
difference 5.551115123125783e-17
individual residuals [1.11022302e-16 1.11022302e-16 5.55111512e-17]
A [[4.47859170e-10 1.11110442e-09]
 [1.11110442e-09 3.19601317e-09]]
det A 1.9681076614389137e-19
indiviudal corrections 6.3766484101125195e-12 -2.436596139445616e-12
measurement [0.58529204 0.74625703 0.31707047]
residuals [1.11022302e-16 1.11022302e

C:\Users\Märten Mikk\AppData\Local\Temp\ipykernel_20260\2704685356.py:6: RuntimeWarning: invalid value encountered in sqrt
  return t - np.sqrt((r**3)/mu)*np.arccos(np.dot(r1,r2)/r**2)


(5.65028567713777e-06,
 2.346171066702998,
 0.03112529196918355,
 0.004926068528437849,
 2.973005583107244,
 -1.5707963267948415)